* Objectif :    
    - Prédire si une personne est infectée en fonction des données cliniques disponibles
* Métrique adaptée choisie ? (ex : F1, Recall...)
    - F1 min 50% et recall min 70%

In [75]:
# Imports standards
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn: model selection
from sklearn.model_selection import (
    GridSearchCV,
    cross_val_score,
    learning_curve,
    train_test_split,
    validation_curve,
)

# Scikit-learn: preprocessing
from sklearn.preprocessing import (
    Binarizer,
    KBinsDiscretizer,
    LabelEncoder,
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    PolynomialFeatures,
    RobustScaler,
    StandardScaler,
)

# Scikit-learn: pipeline & composition
from sklearn.pipeline import make_pipeline, make_union
from sklearn.compose import make_column_selector, make_column_transformer

# Scikit-learn: imputation
from sklearn.impute import SimpleImputer

# Scikit-learn: feature selection
from sklearn.feature_selection import (
    RFECV,
    RFE,
    SelectFromModel,
    SelectKBest,
    VarianceThreshold,
    chi2,
)

# Scikit-learn: models
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.ensemble import (
    AdaBoostClassifier,
    BaggingClassifier,
    GradientBoostingClassifier,
    IsolationForest,
    RandomForestClassifier,
    StackingClassifier,
    VotingClassifier,
)

# Scikit-learn: decomposition
from sklearn.decomposition import PCA

# Scikit-learn: metrics
from sklearn.metrics import (
    confusion_matrix,
    mean_absolute_error,
    mean_squared_error,
    median_absolute_error,
) 
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

## EDA

### Form Analysis (EDA)

* [X]Ai-je identifié la variable cible (target) ?
    - target = data["SARS-Cov-2 exam result	"]

In [76]:
#Importe le dataset covid 19 et affichage des données du dataset
df = pd.read_csv("../data/covid-19.csv", encoding="latin1")
display(df) #Target = SARS-Cov-2 exam result


,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.022340,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5639,ae66feb9e4dc3a0,3,positive,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5640,517c2834024f3ea,17,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5641,5c57d6037fe266d,4,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5642,c20c44766f28291,10,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* [X]Ai-je noté le nombre de lignes et colonnes du dataset ?
    - (5644, 111)

In [77]:
df.shape

(5644, 111)

* [X] Les types de variables sont-ils bien identifiés (numériques, catégorielles...) ? (sns.pie and dtype with value_counts)
    - float64: 70, object: 37, int64: 4

**Tips**:
- (sns.pie and dtype with value_counts)

In [ ]:
#Displays the types of variables in the dataset
print(df.dtypes.value_counts())
plt.pie(df.dtypes.value_counts(), labels=df.dtypes.value_counts().index, autopct='%1.1f%%')

* [X]Ai-je repéré les valeurs manquantes ? 
    - When there are lines in the heatmap, there are relations in the variables
    - 72 columns with missing value percentage over 90%
    - 2 groups: 76% => viral test, 89% taux sanguins

**Tips**:
   - with isna and sns.heatmap
   - IF yes => delete the columns with a too many of Nan values
   - Displays the percentage of missing values for findind groups in data

In [ ]:
### By myself
plt.figure(figsize=(20, 10))
sns.heatmap(df.isna(), cbar=False)
print("Pourcentage total de valeurs manquantes dans le dataset:", (df.isna().sum()).sum() / (df.shape[0] * df.shape[1]))
print("Nombre de colonnes avec plus de 90% de valeurs manquantes:", ((df.isna().sum() / df.shape[0]) > 0.90).sum())
print("Pourcentage de valeurs manquantes par colonne (triées):")
missing_percentages = (df.isna().sum() / df.shape[0]).sort_values(ascending=False)
print(missing_percentages)

### Content analysis (EDA)
##### + nettoyage du df



In [80]:
df = df.loc[:, df.isna().sum()/df.shape[0] < 0.90]
df = df.drop("Patient ID", axis=1)  # Drop the "Patien ID" column as it is not useful for analysis

* Visualisation de la target (histogramme, boxplot) ?
    - 10% positive test => Classes pas équilibrées => utiliser score F1

**Tips**:
- (sns.pie and dtype with value_counts)

In [81]:
df["SARS-Cov-2 exam result"].value_counts(normalize=True)

SARS-Cov-2 exam result
negative    0.901134
positive    0.098866
Name: proportion, dtype: float64

* Significations des variables (Histogrammes de valeurs)?
    - Les courbes sont centrées en 0 avec écrat type = 1 => données standardisées
    - distribution normales pour la plupart
    - virale, variable qualitative binaire, rhinovirus qui semble très élevé

**Tips**:
- Utiliser .select_dtypes() pour sélectionner seulement un type de donnée
- Utiliser unique pour obtenir le type de variable(similaire à value_counts)
- Utiliser pie pour visualiser la distribution de chacune des colonnes

Variables quantitatives

In [ ]:

for col in df.select_dtypes("float"): # Loop through all float columns
    sns.displot(df[col])

In [ ]:
sns.displot(df["Patient age quantile"])

Variables Qualitatives

In [84]:
for col in df.select_dtypes("object"):
    print(f'{col :-<40} {df[col].unique()}')

SARS-Cov-2 exam result------------------ ['negative' 'positive']
Respiratory Syncytial Virus------------- [nan 'not_detected' 'detected']
Influenza A----------------------------- [nan 'not_detected' 'detected']
Influenza B----------------------------- [nan 'not_detected' 'detected']
Parainfluenza 1------------------------- [nan 'not_detected' 'detected']
CoronavirusNL63------------------------- [nan 'not_detected' 'detected']
Rhinovirus/Enterovirus------------------ [nan 'detected' 'not_detected']
Coronavirus HKU1------------------------ [nan 'not_detected' 'detected']
Parainfluenza 3------------------------- [nan 'not_detected' 'detected']
Chlamydophila pneumoniae---------------- [nan 'not_detected' 'detected']
Adenovirus------------------------------ [nan 'not_detected' 'detected']
Parainfluenza 4------------------------- [nan 'not_detected' 'detected']
Coronavirus229E------------------------- [nan 'not_detected' 'detected']
CoronavirusOC43------------------------- [nan 'not_detected

In [ ]:
for col in df.select_dtypes("object"):
    plt.figure()
    df.value_counts(col).plot.pie()
    plt.title(col)

* Relation Target/Variables (histogrammes ...)?
    - blood/target: taux de Monocytes, PLatelets, Leukocytes semblent liés au covid => hypothèse à tester
    - targte/age: quand variable faible, très faible taux de contamination. ATTENTION: on ne connait pas la signification de la variable. Elle pourra être utile pour comparer avec tests sanguins
    - target/viral: double maladie très rare, quand Rhinovirus/Enterovirus positif, covid negatif MAIS pas sûr => à tester, suppression de la variable Parainfluenza 2 car il n'y a aucun cas détécté => on ne peut pas corréler cette maladie avec le covid 19

**Tips**:
- sns.histplot, displot, pairplot
- sns.countplot
- pd.crosstab

Création de sous ensembles positifs/négatifs et blood/viral

In [86]:
positive_df = df[df["SARS-Cov-2 exam result"] == "positive"]
negative_df = df[df["SARS-Cov-2 exam result"] == "negative"]
missing_rate = df.isna().sum() / df.shape[0]
blood_columns = df.columns[(missing_rate < 0.90) & (missing_rate > 0.89)]
viral_columns = df.columns[(missing_rate < 0.77) & (missing_rate > 0.76)]

print(f"blood_columns: {blood_columns.shape}")
print(f"viral_columns: {viral_columns.shape}")

blood_columns: (14,)
viral_columns: (17,)


Relations entre target/blood

In [ ]:
for col in blood_columns:
    plt.figure()
    sns.histplot(positive_df[col], label="positive", kde=True, alpha=0.5, color="blue", stat="density")
    sns.histplot(negative_df[col], label="negative", kde=True, alpha=0.5, color="red", stat="density")
    plt.legend()
    plt.title(f"{col} (positive vs negative)")
    plt.ylabel("Densité")
    plt.show()

Relation target/age

In [ ]:
sns.countplot(x="Patient age quantile", hue="SARS-Cov-2 exam result", data=df)

Relation entre target/viral

In [ ]:
for col in viral_columns:
    plt.figure()
    sns.heatmap(pd.crosstab(df["SARS-Cov-2 exam result"], df[col]), annot=True, fmt="d")

### Analyse plus détaillée

#### Analyse variables/variables
- blood/blood: certaines variables très corrélées (+0.9) => à surveiller plus tard
- blood/age: : corrélation très faible
- viral/viral: Influenza rapid test (A et B) donnent de mauvais résultats => il faudra cerainement les supprimer 
- viral/blood: Relation différente que la relation target/blood. Taux sanguins entre simples malades et covid-19 sont différents
- hospitalisation/blood: intéressant dans le cas où l'on voudrait prédire dans quelle service un patient devrait aller

#### Nan analyse
viral: 1350 données présentes avec une proportion target positif/negatif 92/8, blood: 600 données propo 87/13

**Tips**:
- sns.pairplot
- sns.heatmap
- sns.clustermap
- deux variables que l'on compare avec une autre grâce à histplot

Blood/Blood

In [ ]:
#sns.pairplot(df[blood_columns])
sns.heatmap(df[blood_columns].corr())
sns.clustermap(df[blood_columns].corr())

blood/age

In [ ]:
for col in blood_columns:
    plt.figure()
    sns.lmplot(x="Patient age quantile", y=col, hue="SARS-Cov-2 exam result", data=df) # => pas de corrélation linéaire

In [ ]:
df_blood_columns_and_age = df[blood_columns]
df_blood_columns_and_age["Patient age quantile"] = df["Patient age quantile"]
df_blood_columns_and_age.corr()["Patient age quantile"].sort_values()

Relation entre Influenza et rapid test

In [ ]:
display(pd.crosstab(df["Influenza A"], df["Influenza A, rapid test"])) # Tests peu confiants
display(pd.crosstab(df["Influenza B"], df["Influenza B, rapid test"]))

Reation viral/sanguin

In [120]:
#Création d'une variable "malade" qui compte le nombre de virus détectés pour chaque patient
df["malade"] = np.sum(df[viral_columns[:-2]] == "detected", axis=1)
#Création de deux variables "malade_df" et "non_malade_df" qui contiennent respectivement les patients malades et non malades
malade_df = df[df["malade"] > 0]
non_malade_df = df[df["malade"] == 0]

In [ ]:
for col in blood_columns:
    plt.figure()
    sns.histplot(malade_df[col], label="malade", kde=True, alpha=0.5, color="blue", stat="density")
    sns.histplot(non_malade_df[col], label="non_malade", kde=True, alpha=0.5, color="red", stat="density")
    plt.legend()
    plt.title(f"{col} (positive vs negative)")
    plt.ylabel("Densité")
    plt.show()

Hospitalisation/blood

In [124]:
def hospitalisation(df):
    if df["Patient addmited to regular ward (1=yes, 0=no)"] == 1:
        return "surveillance"
    elif df["Patient addmited to semi-intensive unit (1=yes, 0=no)"] == 1:
        return "semi-intensive"
    elif df["Patient addmited to intensive care unit (1=yes, 0=no)"] == 1:
        return "intensive"
    else:
        return "no_hospitalisation"
    

In [ ]:
df["statut"] = df.apply(hospitalisation, axis=1)
df["statut"].value_counts()
df["statut"].unique()

array(['no_hospitalisation', 'semi-intensive', 'surveillance',
       'intensive'], dtype=object)

In [ ]:
for col in blood_columns:
    plt.figure()
    for cat in df["statut"].unique():
        sns.histplot(df[df["statut"] == cat][col], label=cat, kde=True, alpha=0.5, stat="density")
    plt.legend() 

In [ ]:
# Si on ne travail qu'avec les colonnes sanguines
print(df[blood_columns].count())
# Si on ne travail qu'avec les colonnes virales
print(df[viral_columns].count())